# Fine-tuning FMs for Medical Image Classification

In [1]:
# Core libraries
import torch
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

# Lightning framework
import lightning as pl
from lightning.pytorch.callbacks import (
    ModelCheckpoint, 
    LearningRateMonitor, 
    EarlyStopping, 
    TQDMProgressBar
)
from lightning.pytorch.loggers import WandbLogger

# Custom modules
from models.FMClassifier import FMClassifier
from utils.get_dataloaders import get_dataloader

/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/utils/module.py:399: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  pkg = __import__(module)  # top level module


In [2]:
# Specify which GPU to use
GPU = 0  # Change this to your desired GPU device

os.environ["CUDA_VISIBLE_DEVICES"] = f"{GPU}"
print(f"Using GPU device: {GPU}")

# Weights & Biases configuration
os.environ['WANDB_API_KEY'] = '1fd404a4aa7942f53225bb4b74f219f926325e2a'
os.environ['WANDB_SILENT'] = 'true'         # Suppress W&B verbose output

Using GPU device: 0


## Configuration Setup

Define all hyperparameters and settings for the MedSigLIP fine-tuning process. This centralized configuration approach makes it easy to modify parameters and track different experimental setups.

In [12]:
class Config:
    """
    Configuration class for FM Classifier
    
    This class contains all hyperparameters and settings for training
    a medical image classifier using the SigLIP architecture.
    """
    
    # ========================
    # Label Configuration  
    # ========================
    labels = ['Pneumothorax']  # List of class labels. ex: ['No Finding', 'Pneumothorax']
    
    # ========================
    # Model Architecture
    # ========================
    model_name = "google/siglip2-so400m-patch16-512"  # Pre-trained model names: "StanfordAIMI/XraySigLIP__vit-l-16-siglip-384__webli", "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224", "microsoft/rad-dino", "google/medsiglip-448"     
    freeze_encoder = True                # Whether to freeze the encoder weights
    
    # ========================
    # Training Hyperparameters
    # ========================
    learning_rate = 1e-3                 # Learning rate for AdamW optimizer
    weight_decay = 1e-2                  # Weight decay for AdamW optimizer 
    batch_size = 32                      # Training batch size
    max_epochs = 500                     # Maximum training epochs
    
    # ========================
    # Training Configuration
    # ========================
    precision = 'bf16-mixed'             # Mixed precision training (bf16-mixed/16-mixed/32)
    accelerator = "gpu"                  # Training accelerator (gpu/cpu/tpu)
    num_workers = 4
    
    # ========================
    # Early Stopping & Monitoring
    # ========================
    patience = 5                        # Early stopping patience (epochs)
    min_delta = 0.001                    # Minimum change threshold for early stopping
    
    # ========================
    # Data Configuration
    # ========================
    img_base_path = '/mnt/NAS3/CXR/SIIM_ACR_Pneumothorax/train_png/'  # Base path for images
    input_path = './inputs/input_train_ptx_cla_0.csv'    # Input dataset CSV path
    
    
    # ========================
    # Experiment Tracking
    # ========================
    project = 'FM_evaluation'     # WandB project name
    test_name = 'test'             # Experiment/run name
    entity = 'f10409'                    # WandB entity/username
    
    # ========================
    # Output Configuration
    # ========================
    weight_path = f'./weights/{test_name}'  # Model weights save directory
    
    # ========================
    # Derived Configuration (Auto-computed)
    # ========================
    num_classes = len(labels)            # Number of output classes (auto-computed)
    idx_to_class = {i: label for i, label in enumerate(labels)}  # Index to class mapping (auto-computed)

### Initialize Configuration

Create an instance of the configuration class to use throughout the training pipeline.

In [13]:
# Initialize configuration instance
config = Config()

# Display key configuration parameters
print("=== MedSigLIP Fine-tuning Configuration ===")
print(f"Model: {config.model_name}")
print(f"Classes: {config.labels}")
print(f"Number of classes: {config.num_classes}")
print(f"Batch size: {config.batch_size}")
print(f"Learning rate: {config.learning_rate}")
print(f"Freeze encoder: {config.freeze_encoder}")
print(f"Experiment: {config.project}/{config.test_name}")

=== MedSigLIP Fine-tuning Configuration ===
Model: google/siglip2-so400m-patch16-512
Classes: ['Pneumothorax']
Number of classes: 1
Batch size: 32
Learning rate: 0.001
Freeze encoder: True
Experiment: FM_evaluation/test


## Fine-Tuning

In [14]:
# Load dataset from CSV
input_df = pd.read_csv(config.input_path)

# Display dataset split distribution
print("=== Dataset Split Distribution ===")
print(input_df.Split.value_counts())
print()

# Display class label distribution
print("=== Class Label Distribution ===")
print(input_df[config.labels[0]].value_counts())
print()

# Display dataset overview
print("=== Dataset Overview ===")
print(f"Total samples: {len(input_df)}")
print()

# Show first few rows of the dataset
input_df

=== Dataset Split Distribution ===
Split
Train    2854
Test      952
Valid     952
Name: count, dtype: int64

=== Class Label Distribution ===
Pneumothorax
0    2379
1    2379
Name: count, dtype: int64

=== Dataset Overview ===
Total samples: 4758



,PID,ImageId,DicomPath,Pneumothorax,ImagePath,Fold,Split
0,3018,1.2.276.0.7230010.3.1.4.8323329.442.1517875162...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,0,1.2.276.0.7230010.3.1.4.8323329.442.1517875162...,0,Test
1,1622,1.2.276.0.7230010.3.1.4.8323329.13271.15178752...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.13271.15178752...,0,Test
2,8152,1.2.276.0.7230010.3.1.4.8323329.11965.15178752...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.11965.15178752...,0,Test
3,3852,1.2.276.0.7230010.3.1.4.8323329.12716.15178752...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,0,1.2.276.0.7230010.3.1.4.8323329.12716.15178752...,0,Test
4,2310,1.2.276.0.7230010.3.1.4.8323329.4580.151787518...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.4580.151787518...,0,Test
...,...,...,...,...,...,...,...
4753,9365,1.2.276.0.7230010.3.1.4.8323329.32305.15178751...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.32305.15178751...,4,Train
4754,10684,1.2.276.0.7230010.3.1.4.8323329.3030.151787517...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,0,1.2.276.0.7230010.3.1.4.8323329.3030.151787517...,4,Train
4755,2624,1.2.276.0.7230010.3.1.4.8323329.32355.15178751...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.32355.15178751...,4,Train
4756,2271,1.2.276.0.7230010.3.1.4.8323329.1632.151787516...,/dicom-images-train/1.2.276.0.7230010.3.1.2.83...,1,1.2.276.0.7230010.3.1.4.8323329.1632.151787516...,4,Train


### Data Loader Creation

In [15]:
# Create data loaders for all dataset splits
train_dataloader, valid_dataloader, test_dataloader = get_dataloader(input_df, config)

# Display data loader information
print("=== Data Loaders Created ===")
print(f"Training batches: {len(train_dataloader)}")
print(f"Validation batches: {len(valid_dataloader)}")
print(f"Test batches: {len(test_dataloader)}")
print(f"Batch size: {config.batch_size}")

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
Processing part: 100%|██████████| 356/356 [00:00<00:00, 31760.84it/s]


Processing part: 100%|██████████| 357/357 [00:00<00:00, 21313.31it/s]

Processing part: 100%|██████████| 119/119 [00:00<00:00, 30821.43it/s]



Processing part: 100%|██████████| 119/119 [00:00<00:00, 19300.19it/s]



Processing part: 100%|██████████| 119/119 [00:00<00:00, 29434.58it/s]


Processing part: 100%|██████████| 119/119 [00:00<00:00, 19608.02it/s]

Processing part: 100%|██████████| 119/119 [00:00<00:00, 19418.07it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


=== Data Loaders Created ===
Training batches: 89
Validation batches: 30
Test batches: 30
Batch size: 32


In [16]:
for b in train_dataloader:
    print(b)
    break

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/transform.py", line 141, in apply_transform
    return _apply_transform(transform, data, unpack_items, lazy, overrides, log_stats)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/transform.py", line 98, in _apply_transform
    return transform(data, lazy=lazy) if isinstance(transform, LazyTrait) else transform(data)
                                                                               ^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/io/dictionary.py", line 162, in __call__
    data = self._loader(d[key], reader)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40

In [ ]:
b['img'].shape

In [17]:
import matplotlib.pyplot as plt
plt.imshow(b['img'][30,0,:,:])

NameError: name 'b' is not defined

### Model Initialization

Initialize the MedSigLIP classifier with the specified configuration. The model will use:

- Pre-trained weights from `google/medsiglip-448`
- Frozen encoder (if specified) to retain pre-trained features
- Custom classification head for the target classes

In [18]:
# Initialize MedSigLIP classifier model
model = FMClassifier(
    config=config
)

### Training Callbacks and Logger Setup

Configure PyTorch Lightning callbacks and logging for monitoring and controlling the training process:

#### Callbacks:
- **Learning Rate Monitor**: Tracks learning rate 
- **Model Checkpoint**: Saves the best model based on validation loss
- **Early Stopping**: Prevents overfitting by stopping when validation loss plateaus
- **Progress Bar**: Provides visual training progress

#### Logging:
- **Weights & Biases**: Tracks metrics, hyperparameters, and training progress

In [19]:
# Learning rate monitoring
lr_monitor = LearningRateMonitor(logging_interval="step")

# Model checkpointing - save best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    dirpath=f"{config.weight_path}",
    filename=f'{config.test_name}_{{epoch}}_{{val_loss:0.4F}}',
    monitor="val_loss",
    mode="min",
    save_last=False,
    save_top_k=1
)

# Early stopping to prevent overfitting
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=config.min_delta,
    patience=config.patience,
    verbose=False,
    mode='min'
)

# Weights & Biases logger for experiment tracking
wandb_logger = WandbLogger(
    save_dir=f"{config.weight_path}",
    name=f'{config.test_name}',
    project=config.project,
    entity=config.entity,
    offline=False,
    log_model=False,
    config={"Creator": "HITI"}
)

# Progress bar for training visualization
progress_bar = TQDMProgressBar()

# Display callback configuration
print("=== Training Configuration ===")
print(f"Checkpoint directory: {config.weight_path}")
print(f"Early stopping patience: {config.patience} epochs")
print(f"Min delta for early stopping: {config.min_delta}")
print(f"W&B project: {config.project}")
print(f"Experiment name: {config.test_name}")

=== Training Configuration ===
Checkpoint directory: ./weights/test
Early stopping patience: 5 epochs
Min delta for early stopping: 0.001
W&B project: FM_evaluation
Experiment name: test


### Trainer Initialization

Configure the PyTorch Lightning Trainer with all training parameters, callbacks, and logging setup. 

In [20]:
# Instantiate PyTorch Lightning trainer
trainer = pl.Trainer(
    gradient_clip_val=1.0,                    # Clip gradients to prevent exploding gradients
    callbacks=[progress_bar, lr_monitor, checkpoint_callback, early_stop_callback],
    logger=wandb_logger,                      # W&B logger for experiment tracking
    precision=config.precision,               # Mixed precision training (bf16-mixed)
    accelerator=config.accelerator,           # Use GPU acceleration
    devices=1,                                # Single GPU training
    log_every_n_steps=1,                      # Log metrics every step
    default_root_dir=config.weight_path,      # Directory for trainer outputs
    max_epochs=config.max_epochs              # Maximum training epochs
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### Model Training

Begin the fine-tuning process using PyTorch Lightning's `fit` method.

**Note**: Monitor the progress through the displayed metrics and W&B dashboard.

In [21]:
# train the model

# from monai.data.meta_tensor import MetaTensor

# # Add all necessary safe globals
# torch.serialization.add_safe_globals([
#     np.core.multiarray._reconstruct,
#     np.ndarray,
#     np.dtype,
#     np.core.multiarray.scalar,
#     MetaTensor,  # This is the missing one!
# ])

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

You are using a CUDA device ('NVIDIA L40S') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Checkpoint directory /mnt/NAS3/homes/fli40/FM_evaluation/weights/test exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | vision_encoder | SiglipVisionModel | 428 M  | eval 
1 | classifier     | Sequential        | 1.2 K  | train
-------------------------------------------------------------
1.2 K     Trainable params
428 M     Non-trainable params
428 M 

Encoder frozen. Using single LR: 0.001


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/transform.py", line 141, in apply_transform
    return _apply_transform(transform, data, unpack_items, lazy, overrides, log_stats)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/transform.py", line 98, in _apply_transform
    return transform(data, lazy=lazy) if isinstance(transform, LazyTrait) else transform(data)
                                                                               ^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40/FM_evaluation/.venv/lib/python3.12/site-packages/monai/transforms/io/dictionary.py", line 162, in __call__
    data = self._loader(d[key], reader)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/NAS3/homes/fli40